# Bidding

This is a tutorial for how to use the bidding engine

In [1]:
import os
import sys
sys.path.append('../../src')

from nn.models import Models
from bots import BotBid
from util import hand_to_str
from sample import Sample
import conf
import numpy as np

np.set_printoptions(precision=2, suppress=True, linewidth=200)
np.random.seed(42)


Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
models = Models.from_conf(conf.load('../Conf/UCBC2024.conf'),'..')   # loading neural networks
sampler = Sample.from_conf(conf.load('../Conf/UCBC2024.conf'), False)  # Load sampling strategies


INFO:tensorflow:Restoring parameters from ..\Models/bidding-3233000
INFO:tensorflow:Restoring parameters from ..\Models/binfo-808000
INFO:tensorflow:Restoring parameters from ..\Models/lead-1000000
INFO:tensorflow:Restoring parameters from ..\Models/lead-1000000
INFO:tensorflow:Restoring parameters from ..\Models/lr3-1000000
INFO:tensorflow:Restoring parameters from ..\Models/lefty-1000000
INFO:tensorflow:Restoring parameters from ..\Models/dummy-920000
INFO:tensorflow:Restoring parameters from ..\Models/righty-1000000
INFO:tensorflow:Restoring parameters from ..\Models/decl-1000000


### Running through an example

In [3]:
# East deals, EW vulnerable.
vuln_ns, vuln_ew = False, False

# you sit West and hold:
hand = 'AQ8.KQ94.JT.KJ52'

# the auction goes:
# (a few words about 'PAD_START':
# the auction is padded to dealer North
# if North is not dealer, than we have to put in a 'PAD_START' for every seat that was skipped
# if East deals we have one pad (because North is skipped)
# if South deals we have two pads (because North and East are skipped)
# etc.)
auction = ["PAD_START", "PAD_START", "PAD_START", "PASS"]
bot_bid = BotBid([vuln_ns, vuln_ew], hand, models, sampler, False)

bid = bot_bid.bid(auction)
bid.to_dict()['candidates']

# what's your bid?

[{'call': '1N', 'insta_score': 0.9994}]

In [4]:
auction = ["PAD_START", "PAD_START", "PAD_START", "PASS", "1N", "PASS", "3S","PASS"]
bid = bot_bid.bid(auction)

bid.to_dict()['candidates']

[{'call': '4H',
  'insta_score': 0.7903,
  'expected_score': 349,
  'adjustment': 40},
 {'call': '3N', 'insta_score': 0.1586, 'expected_score': 268, 'adjustment': 8}]

in the output above:
- `insta_score` reflects what the neural network prefers to bid
- `expected_score` is based on what you expect to score on difference samples (for example, 70% of the time you are making and 30% of the time you are going one off, then the expected score is 0.7 * 620 - 0.3 * 100)

### Samples consistent with the auction so far

Above we said that we are computig expected scores on samples consistent with the auction so far.

How do we get these samples?

We get some samples in the response we get from the bot (i.e `bid.samples`)

In [5]:
bid.samples

['AQ8.KQ9x.JT.KJxx JT9xx.Axx.xx.9xx x.JT8x.AQ9x.A8xx Kxxx.xx.K8xxx.QT 0.99088 P-1N-P-3S-P-4H-P-P-P (10.6)  P-1N-P-3S-P-3N-P-P-P (9.1) ',
 'AQ8.KQ9x.JT.KJxx J9xxx.J8x.xx.AQx .Axxx.AKQ9x.T9xx KTxxx.Tx.8xxx.8x 0.98844 P-1N-P-3S-P-4H-P-P-P (11.6)  P-1N-P-3S-P-3N-P-P-P (10.4) ',
 'AQ8.KQ9x.JT.KJxx Txxxx.JT8x.Axx.T 9.Axxx.K98x.Qxxx KJxx.x.Qxxx.A98x 0.98477 P-1N-P-3S-P-4H-P-P-P (9.0)  P-1N-P-3S-P-3N-P-P-P (8.7) ',
 'AQ8.KQ9x.JT.KJxx Kxxxx.Jx.K9xxx.x x.A8xx.xxx.AQxxx JT9x.Txx.AQ8.T98 0.98457 P-1N-P-3S-P-4H-P-P-P (10.9)  P-1N-P-3S-P-3N-P-P-P (9.2) ',
 'AQ8.KQ9x.JT.KJxx KT9xx.Jxx.Q98x.Q x.ATxx.Kxxx.A98x Jxxx.8x.Axx.Txxx 0.97891 P-1N-P-3S-P-4H-P-P-P (11.0)  P-1N-P-3S-P-3N-P-P-P (9.5) ',
 'AQ8.KQ9x.JT.KJxx K9xx.Txx.9x.Qxxx T.Jxxx.AKxx.AT98 Jxxxx.A8.Q8xxx.x 0.97796 P-1N-P-3S-P-4H-P-P-P (11.2)  P-1N-P-3S-P-3N-P-P-P (10.0) ',
 'AQ8.KQ9x.JT.KJxx Jxxxx.Tx.Kxx.A9x x.AJxx.Axxx.8xxx KT9x.8xx.Q98x.QT 0.97521 P-1N-P-3S-P-4H-P-P-P (10.4)  P-1N-P-3S-P-3N-P-P-P (8.6) ',
 'AQ8.KQ9x.JT.KJxx JTxx.A8x.9x.AQ9x x.Tx

each row is one sample. the hands are in the order: N E S W

(West always has the same cards because it's your hand and it's known to you)

In [6]:
# we got some samples above. if we want more, we can always get more

sample_hands, sorted_score, p_hcp, p_shp = bot_bid.sample_hands(["PAD_START", "PAD_START", "PAD_START", "PASS", "1N", "PASS", "3S","PASS"])

for i in range(sample_hands.shape[0]):
    print(f'{hand_to_str(sample_hands[i, 0, :])} {hand_to_str(sample_hands[i, 1, :])} {hand_to_str(sample_hands[i, 2, :])} {hand_to_str(sample_hands[i, 3, :])}')

AQ8.KQ9x.JT.KJxx KJ9xx.8xxx.xx.Tx x.AJxx.98xx.AQxx Txxx.T.AKQxx.98x
AQ8.KQ9x.JT.KJxx KTxxxx.x.A9xx.9x J.ATxx.Q8x.QT8xx 9xx.J8xx.Kxxx.Ax
AQ8.KQ9x.JT.KJxx T9xx.AJx.AQ9x.T8 x.T8xx.Kxxx.AQ9x KJxxx.xx.8xx.xxx
AQ8.KQ9x.JT.KJxx KJ9.Jxx.KQ8.QTxx T.Axxx.A9x.A98xx xxxxxx.T8.xxxxx.
AQ8.KQ9x.JT.KJxx KJx.Jxx.K8xx.Q9x x.AT8x.AQxx.ATxx T9xxxx.xx.9xx.8x
AQ8.KQ9x.JT.KJxx KJxxxx.Tx.8xxx.Q x.AJxx.A9.AT9xxx T9x.8xx.KQxxx.8x
AQ8.KQ9x.JT.KJxx xxxxx.xxx.8x.A9x J.T8xx.AKQxxx.Qx KT9x.AJ.9xx.T8xx
AQ8.KQ9x.JT.KJxx KJTx.T.Qxxxx.T9x x.8xxx.A98x.AQxx 9xxxx.AJxx.Kx.8x
AQ8.KQ9x.JT.KJxx Txxx.Txxx.xx.ATx x.AJ8x.AQxxx.Q8x KJ9xx.x.K98x.9xx
AQ8.KQ9x.JT.KJxx xxxx.8xx.Axx.A9x K.AJxx.K98x.Q8xx JT9xx.Tx.Qxxx.Tx
AQ8.KQ9x.JT.KJxx xxxx.Txx.Kxx.Q98 T.AJxx.A8x.Txxxx KJ9xx.8x.Q9xxx.A
AQ8.KQ9x.JT.KJxx T9xxx.A8x.Qx.T9x K.Txxx.K8xxx.AQx Jxxx.Jx.A9xx.8xx
